In [ ]:
import os
from pydna.dseqrecord import Dseqrecord
from pydna.assembly2 import (
    golden_gate_assembly,
    gibson_assembly,
    ligation_assembly,
    gateway_assembly,
    homologous_recombination_integration,
    homologous_recombination_excision,
    cre_lox_integration,
    cre_lox_excision,
    pcr_assembly,
)

from Bio.Restriction import BsaI, EcoRI, SalI
from pydna.parsers import parse_snapgene
from pydna.opencloning_models import CloningStrategy
from pydna.primer import Primer
from Bio.Seq import reverse_complement


## Golden Gate assembly (BsaI)
This section assembles three inserts into a circular vector using Golden Gate with BsaI and exports the design as JSON.


In [ ]:
insert1 = Dseqrecord("GGTCTCAattaAAAAAttaaAGAGACC", name="insert1")
insert2 = Dseqrecord("GGTCTCAttaaCCCCCatatAGAGACC", name="insert2")
insert3 = Dseqrecord("GGTCTCAatatGGGGGccggAGAGACC", name="insert3")

vector = Dseqrecord("TTTTattaAGAGACCTTTTTGGTCTCAccggTTTT", circular=True, name="vector")

product, *_ = golden_gate_assembly(
    [insert1, insert2, insert3, vector], [BsaI], circular_only=True
)
product.name = "product"

cs = CloningStrategy.from_dseqrecord(product)

with open("golden_gate.json", "w") as f:
    f.write(cs.model_dump_json())

CloningStrategy.model_validate(cs.model_dump())

product.history()


╙── product (Dseqrecord(o39))
    └─╼ RestrictionAndLigationSource
        ├─╼ insert1 (Dseqrecord(-27))
        ├─╼ insert2 (Dseqrecord(-27))
        ├─╼ insert3 (Dseqrecord(-27))
        └─╼ vector (Dseqrecord(o35))


## Gibson assembly
This section assembles three linear fragments using Gibson assembly and serializes the resulting product to JSON.


In [ ]:
fragments = [
    Dseqrecord("TTTTacgatAAtgctccCCCC", circular=False, name="fragment1"),
    Dseqrecord("CCCCtcatGGGG", circular=False, name="fragment2"),
    Dseqrecord("GGGGatataTTTT", circular=False, name="fragment3"),
]

product, *_ = gibson_assembly(fragments, limit=4)
product.name = "product"

cs = CloningStrategy.from_dseqrecord(product)
with open("gibson.json", "w") as f:
    f.write(cs.model_dump_json())

print(product.history())


╙── product (Dseqrecord(o34))
    └─╼ GibsonAssemblySource
        ├─╼ fragment1 (Dseqrecord(-21))
        ├─╼ fragment2 (Dseqrecord(-12))
        └─╼ fragment3 (Dseqrecord(-13))
None


## Restriction, then ligation assembly (EcoRI/SalI)
This section cuts a template with EcoRI and SalI, ligates compatible fragments, and exports the result as JSON.


In [ ]:
a = Dseqrecord("aaGAATTCccGTCGACaa")

c, d, e = a.cut(EcoRI, SalI)

product, *_ = ligation_assembly([c, d, e])

a.name = "a"
c.name = "c"
d.name = "d"
e.name = "e"
product.name = "product"

cs = CloningStrategy.from_dseqrecord(product)

with open("restriction_ligation.json", "w") as f:
    f.write(cs.model_dump_json())

print(f.history())

AttributeError: '_io.TextIOWrapper' object has no attribute 'history'

### Gateway assembly (BP and LR)
This section creates an Entry vector via BP reaction, then generates an Expression clone via LR reaction using test SnapGene files.


In [ ]:
test_files = "tests"

line = "pDONRtm221.dna	pcr_product-attP1_1-attP2_1.dna	entry-attP1_1-attP2_1.dna	pET-53-DESTtm.dna	expression-attP1_1-attP2_1.dna".split(
    "\t"
)
backbone, pcr_product, _, backbone_expression, _ = line
backbone = parse_snapgene(os.path.join("tests/gateway_manual_cloning/" + backbone))[0]
pcr_product = parse_snapgene(
    os.path.join("tests/gateway_manual_cloning/" + pcr_product)
)[0]
backbone_expression = parse_snapgene(
    os.path.join("tests/gateway_manual_cloning/" + backbone_expression)
)[0]

# Works with the right reaction
entry_vector = gateway_assembly(
    [backbone, pcr_product], "BP", multi_site_only=True, circular_only=True
)[0]

expression_clone, _ = gateway_assembly(
    [backbone_expression, entry_vector], "LR", multi_site_only=True, circular_only=True
)

cs = CloningStrategy.from_dseqrecord(expression_clone)
with open("gateway.json", "w") as f:
    f.write(cs.model_dump_json())


## Homologous recombination integration
This section performs homologous recombination integration of two inserts into a genome and exports the result as JSON.

In [ ]:
homology = "AAGTCCGTTCGTTTTACCTG"
genome = Dseqrecord(f"aaaaaa{homology}ccccc{homology}aaaaaa")
insert = Dseqrecord(f"{homology}gggg{homology}")
integration_product, *_ = homologous_recombination_integration(genome, [insert], 20)

excision_product, *_ = homologous_recombination_excision(integration_product, 20)

cs = CloningStrategy.from_dseqrecord(excision_product)
with open("homologous_recombination.json", "w") as f:
    f.write(cs.model_dump_json())


## Cre-lox integration
This section performs Cre-lox integration , then excision.

In [ ]:
from pydna.cre_lox import LOXP_SEQUENCE
a = Dseqrecord(f"cccccc{LOXP_SEQUENCE}aaaaa")
b = Dseqrecord(f"{LOXP_SEQUENCE}bbbbb", circular=True)
integration_product, *_ = cre_lox_integration(a, [b])
excision_product, *_ = cre_lox_excision(integration_product)

cs = CloningStrategy.from_dseqrecord(excision_product)
with open("cre_lox.json", "w") as f:
    f.write(cs.model_dump_json())



## PCR assembly
This section performs PCR assembly and exports the result as JSON.


In [ ]:
primer1 = Primer("ACGTACGT")
primer2 = Primer(reverse_complement("GCGCGCGC"))

seq = Dseqrecord("ccccACGTACGTAAAAAAGCGCGCGCcccc")

prod, *_ = pcr_assembly(seq, primer1, primer2, limit=8)

cs = CloningStrategy.from_dseqrecord(prod)

with open("pcr.json", "w") as f:
    f.write(cs.model_dump_json())